In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("circle_chart_with_trends(20241002).csv")
data.head()

,date,Rank,Title,Artist,Production,Weekly Views,Genre,Runtime,New_Artist,ky_rank,n_score,g_score
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,3:00,악뮤,NaN,94.32015,100.0
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,3:25,악뮤,NaN,94.32015,100.0
2,2023-09-24,3,"Smoke (Prod. Dynamicduo, Padi)","다이나믹 듀오 (Dynamic Duo), 이영지",Stone Music Entertainment,4473459.0,가요 / 랩/힙합,3:30,"다이나믹 듀오 , 이영지",NaN,NaN,0
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,2:35,뉴진스,NaN,10.55509,6.0
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,3:04,정국,NaN,61.50958,45.0


In [3]:
data.isnull().sum()

date                0
Rank                0
Title               0
Artist              0
Production          0
Weekly Views     2671
Genre               5
Runtime             5
New_Artist          0
ky_rank         10224
n_score           837
g_score            56
dtype: int64

## Genre, Runtime 결측치 제거

In [4]:
data['Genre'] = data['Genre'].fillna('가요 / R&B/소울')
data['Runtime'] = data['Runtime'].fillna('03:09')

In [5]:
data['Artist'] = data['Artist'].replace('싱어게인3 49호 가수', '소수빈')

In [6]:
data.isnull().sum()

date                0
Rank                0
Title               0
Artist              0
Production          0
Weekly Views     2671
Genre               0
Runtime             0
New_Artist          0
ky_rank         10224
n_score           837
g_score            56
dtype: int64

In [7]:
data.isnull().sum()

date                0
Rank                0
Title               0
Artist              0
Production          0
Weekly Views     2671
Genre               0
Runtime             0
New_Artist          0
ky_rank         10224
n_score           837
g_score            56
dtype: int64

## 해야 할 일
- 1주 전, 2주 전, 3주 전 Rank 값을 Rank_lag_1, Rank_lag_2, Rank_lag_3 컬럼으로 저장 (shift 사용했던 걸로 기억)
- 만약 Rank_lag에 결측치가 있으면 201로 해주세요
- Weekly Views 결측치는 0으로 대체
- n_score, g_score는 해당되는 row가 소속된 날짜의 중앙값 또는 0으로 대체
- date와 Rank가 중복된 컬럼이 있으면 가장 위 컬럼만 남기고 나머지는 제거

다 했으면
```data.isnull().sum()```
명령어 실행해서 모든 컬럼이 0인지 확인

## ▽▼▽ 아래 부분에서부터 일하시오 ▽▼▽

In [8]:
data.head()

,date,Rank,Title,Artist,Production,Weekly Views,Genre,Runtime,New_Artist,ky_rank,n_score,g_score
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,3:00,악뮤,NaN,94.32015,100.0
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,3:25,악뮤,NaN,94.32015,100.0
2,2023-09-24,3,"Smoke (Prod. Dynamicduo, Padi)","다이나믹 듀오 (Dynamic Duo), 이영지",Stone Music Entertainment,4473459.0,가요 / 랩/힙합,3:30,"다이나믹 듀오 , 이영지",NaN,NaN,0
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,2:35,뉴진스,NaN,10.55509,6.0
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,3:04,정국,NaN,61.50958,45.0


In [12]:
# 1. 1주 전, 2주 전, 3주 전 Rank 값을 Rank_lag_1, Rank_lag_2, Rank_lag_3 컬럼으로 저장
data['Rank_lag_1'] = data['Rank'].shift(1)
data['Rank_lag_2'] = data['Rank'].shift(2)
data['Rank_lag_3'] = data['Rank'].shift(3)

# 2. Rank_lag 컬럼의 결측치는 201로 대체
data[['Rank_lag_1', 'Rank_lag_2', 'Rank_lag_3']] = data[['Rank_lag_1', 'Rank_lag_2', 'Rank_lag_3']].fillna(201)

# 3. Weekly Views 결측치는 0으로 대체
data['Weekly Views'] = data['Weekly Views'].fillna(0)

# 4. n_score와 g_score 컬럼을 숫자로 변환 (변환할 수 없는 값은 NaN으로 설정)
data['n_score'] = pd.to_numeric(data['n_score'], errors='coerce')
data['g_score'] = pd.to_numeric(data['g_score'], errors='coerce')

# n_score, g_score 결측치는 해당 날짜의 중앙값 또는 0으로 대체
for col in ['n_score', 'g_score']:
    data[col] = data.groupby('date')[col].transform(lambda x: x.fillna(x.median() if not x.median() is None else 0))

# 5. date와 Rank가 중복된 경우, 첫 번째 행만 남기고 제거
data = data.drop_duplicates(subset=['date', 'Rank'], keep='first')

C:\Users\user\AppData\Local\Temp\ipykernel_12124\1212455883.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Rank_lag_1'] = data['Rank'].shift(1)
C:\Users\user\AppData\Local\Temp\ipykernel_12124\1212455883.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Rank_lag_2'] = data['Rank'].shift(2)
C:\Users\user\AppData\Local\Temp\ipykernel_12124\1212455883.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [13]:
data.head()

,date,Rank,Title,Artist,Production,Weekly Views,Genre,Runtime,New_Artist,ky_rank,n_score,g_score,Rank_lag_1,Rank_lag_2,Rank_lag_3
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,3:00,악뮤,NaN,94.32015,100.0,201.0,201.0,201.0
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,3:25,악뮤,NaN,94.32015,100.0,1.0,201.0,201.0
2,2023-09-24,3,"Smoke (Prod. Dynamicduo, Padi)","다이나믹 듀오 (Dynamic Duo), 이영지",Stone Music Entertainment,4473459.0,가요 / 랩/힙합,3:30,"다이나믹 듀오 , 이영지",NaN,43.00210,0.0,2.0,1.0,201.0
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,2:35,뉴진스,NaN,10.55509,6.0,3.0,2.0,1.0
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,3:04,정국,NaN,61.50958,45.0,4.0,3.0,2.0


In [14]:
data.isnull().sum()

date               0
Rank               0
Title              0
Artist             0
Production         0
Weekly Views       0
Genre              0
Runtime            0
New_Artist         0
ky_rank         9799
n_score            0
g_score            0
Rank_lag_1         0
Rank_lag_2         0
Rank_lag_3         0
dtype: int64

In [15]:
data.to_csv("DBdata.csv")